In [21]:
print("Work started")

Work started


In [22]:
!pip install -q --no-index --no-deps /kaggle/input/pycocotools-206/wheels/*.whl
from pycocotools import _mask as coco_mask


In [23]:
!pip install -q --no-index --no-deps /kaggle/input/myupload/*.whl
import segmentation_models_pytorch
from segmentation_models_pytorch.encoders import get_preprocessing_fn

ERROR: Cannot uninstall typing-extensions 4.5.0, RECORD file not found. You might be able to recover from this via: 'pip install --force-reinstall --no-deps typing-extensions==4.5.0'.


In [24]:
#from IPython.display import FileLink
#FileLink(r'segmentation_models_pytorch.zip')

In [25]:
#!pip install pycocotools==2.0.6
#!pip install segmentation_models_pytorch
import numpy as np
from typing import Text
import base64
import numpy as np
import pandas as pd
import zlib
import glob
import os
from pathlib import Path
import matplotlib.pyplot as plt
import tifffile


import torch
import torch.nn as nn


In [26]:
def encode_binary_mask(mask: np.ndarray) -> Text:
  """Converts a binary mask into OID challenge encoding ascii text."""

  # check input mask --
  if mask.dtype != bool:
    raise ValueError(
        "encode_binary_mask expects a binary mask, received dtype == %s" %
        mask.dtype)

  mask = np.squeeze(mask)
  if len(mask.shape) != 2:
    raise ValueError(
        "encode_binary_mask expects a 2d mask, received shape == %s" %
        mask.shape)

  # convert input mask to expected COCO API input --
  mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
  mask_to_encode = mask_to_encode.astype(np.uint8)
  mask_to_encode = np.asfortranarray(mask_to_encode)

  # RLE encode mask --
  encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

  # compress and base64 encoding --
  binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
  base64_str = base64.b64encode(binary_str)
  return base64_str

In [27]:
data_directory = "/kaggle/input/hubmap-hacking-the-human-vasculature"

In [28]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [29]:
import segmentation_models_pytorch as smp

class UNetInstanceSegmentation(nn.Module):
    def __init__(self, num_classes=1, num_channels=3):
        super(UNetInstanceSegmentation, self).__init__()
        self.model = smp.Unet(
            encoder_name="resnet34",  # You can choose different encoder backbones if desired.
            encoder_weights="imagenet",
            in_channels=num_channels,
            classes=num_classes,
            activation='sigmoid'
        )

    def forward(self, x):
        return self.model(x)

    def predict(self, x):
        with torch.no_grad():
            y_pred = self.forward(x)
        return (y_pred > 0.5) * 1.0


In [30]:
#model_path = f"/kaggle/input/test-stuff/models/onereg_20min"

# model.load_state_dict(torch.load(model_path))

#model = UNetInstanceSegmentation()
#model.load_state_dict(torch.load(model_path))


In [31]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [32]:
#save_whole_model = UNetInstanceSegmentation().to(device)
#save_whole_model.load_state_dict(torch.load('/kaggle/input/myupload2/sleepmodel_unetinstancesegmentation_preprocessing',
#                                            map_location=torch.device(device)), strict=False)
#torch.save(save_whole_model, "sleep_wholemodel")

In [33]:
#from IPython.display import FileLink
#FileLink(r'sleep_wholemodel')

In [34]:
model = torch.load("/kaggle/input/myupload3/sleep_wholemodel", map_location=torch.device(device))
#model = torch.load("/kaggle/input/test-stuff/models/onereg_20min")

print(type(model))

<class '__main__.UNetInstanceSegmentation'>


In [35]:
def convert_multiple_masks(masks: list[np.ndarray], confidence_values: list[int]) -> Text:
    strings = []
    for mask, confidence in zip(masks, confidence_values):
        strings.append(f"0 {confidence:.1f} {encode_binary_mask(mask).decode('utf-8')}")

    return ' '.join(strings)

In [36]:
def instance_confidence(masks, conf_matrixes) -> list:
    conf_values = []
    for mask, conf_matrix in zip(masks, conf_matrixes):
        num, denom = 0, 0
        for i in range(mask.shape[0]):
            for j in range(mask.shape[1]):
                if mask[i][j] == 1:
                    num += conf_matrix[i][j]
                    denom += 1
        conf_values.append(num/denom)
    return conf_values

In [20]:
preprocessing_fn = get_preprocessing_fn('resnet34', pretrained='imagenet')


In [45]:
submission_df = pd.DataFrame(columns=['id', 'height', 'width', 'prediction_string'])

tif_files = glob.glob(os.path.join(f'{data_directory}/test', '*.tif'))
rows = []
for file_path in tif_files:
    
    # Preparation of image
    image_id = Path(file_path).stem
    image = tifffile.imread(file_path)
    height, width = image.shape[0], image.shape[1]
    image.resize(512, 512, 3)
    
    # Preprocessing
    #image = preprocessing_fn(image)
    image = image / 255
    
    
    inp = torch.tensor(image).to(device) # imread returns (512, 512, 3)
    inp = inp.reshape(1, 3, 512, 512) # BATCH, CHANNELS, SIZEY, SIZEX
    inp = inp.float() # from float64 to float32

    # Prediction + post preparation
    with torch.no_grad():
        conf_matrix = model.forward(inp).cpu().numpy().reshape(512, 512)
    predicted = model.predict(inp).cpu().numpy().reshape(512, 512).astype(bool)

    # plt.imshow(predicted.numpy().reshape(512, 512, 1), vmax=1, vmin=0, cmap="gray")
    
    # Mask separation - TODO
    masks = []
    masks.append(predicted)
    #

    # Confidence
    conf_matrixes = []
    conf_matrixes.append(conf_matrix)

    # confidence =  sum of all pixels confidence over treshold divided by the number of  pixels confidence over treshold
    confidence_values = instance_confidence(masks, conf_matrixes)
    #print(confidence_values)
    #print(conf_matrixes)

    # Prediction string
    prediction_str = convert_multiple_masks(masks, confidence_values)  

    # Add row to list  
    new_row = {
        'id': image_id,
        'height': height,
        'width': width,
        'prediction_string': prediction_str
    }
    rows.append(new_row)


res = submission_df.from_dict(rows)
res.set_index("id", inplace=True)
#res.reset_index(inplace=True)

res.head()


,height,width,prediction_string
id,,,
72e40acccadf,512,512,0 1.0 eNptVem2oroSfqUMRAJ791nnrt0SDBlkEhVwxBGF...


In [38]:
#print(res)
res.to_csv(f'submission.csv')